In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install -q pyspark

In [3]:

import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from scipy.sparse import dok_matrix
from pyspark.ml.linalg import Vectors
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import math
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*


-rw-r--r-- 1 root root 247882 Mar  9 16:20 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *


In [6]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("try1") \
    .config("spark.driver.memory", "32g") \
    .config("spark.driver.maxResultSize", "32g") \
    .config("spark.num.executors", "4") \
    .config("spark.executor.memory", "5950m") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/09 17:54:28 INFO SparkEnv: Registering MapOutputTracker
24/03/09 17:54:28 INFO SparkEnv: Registering BlockManagerMaster
24/03/09 17:54:28 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/03/09 17:54:28 INFO SparkEnv: Registering OutputCommitCoordinator


In [36]:
spark.stop()

In [7]:

# Put your bucket name below and make sure you can access it without an error
bucket_name = 'proj_479_187' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name[0] == 'm':
        paths.append(full_path+b.name)


In [8]:
paths

['gs://proj_479_187/multistream10_preprocessed.parquet',
 'gs://proj_479_187/multistream11_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream11_preprocessed.parquet',
 'gs://proj_479_187/multistream12_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream12_preprocessed.parquet',
 'gs://proj_479_187/multistream13_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream13_preprocessed.parquet',
 'gs://proj_479_187/multistream14_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream14_preprocessed.parquet',
 'gs://proj_479_187/multistream15_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream15_part3_preprocessed.parquet',
 'gs://proj_479_187/multistream15_preprocessed.parquet',
 'gs://proj_479_187/multistream16_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream16_part3_preprocessed.parquet',
 'gs://proj_479_187/multistream16_preprocessed.parquet',
 'gs://proj_479_187/multistream17_part2_preprocessed.parquet',
 'gs://proj_479_187/multistream17_

In [9]:

parquetFile = spark.read.parquet(*paths)
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd


In [10]:
doc_text_pairs=doc_text_pairs.repartition(104)
doc_title_pairs=doc_title_pairs.repartition(104)
doc_anchor_pairs=doc_anchor_pairs.repartition(104)

In [11]:
text_splits = doc_text_pairs.randomSplit([1] * 50)

PythonRDD[39] at RDD at PythonRDD.scala:53

In [12]:
# -*- coding: utf-8 -*-
"""Untitled13.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1-96zO3pwOgfY2ST2rhlwseOZ8R8V2qsR
"""

import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
from time import time
from pathlib import Path
import pickle
from google.cloud import storage
from collections import defaultdict
from contextlib import closing
import math

PROJECT_ID = 'projectgcp-415908'
def get_bucket(bucket_name):
    return storage.Client(PROJECT_ID).bucket(bucket_name)

def _open(path, mode, bucket=None):
    if bucket is None:
        return open(path, mode)
    return bucket.blob(path).open(mode)

# Let's start with a small block size of 30 bytes just to test things out.
BLOCK_SIZE = 1999998

class MultiFileWriter:
    """ Sequential binary writer to multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, name, bucket_name=None):
        self._base_dir = Path(base_dir)
        self._name = name
        self._bucket = None if bucket_name is None else get_bucket(bucket_name)
        self._file_gen = (_open(str(self._base_dir / f'{name}_{i:03}.bin'),
                                'wb', self._bucket)
                          for i in itertools.count())
        self._f = next(self._file_gen)

    def write(self, b):
        locs = []
        while len(b) > 0:
            pos = self._f.tell()
            remaining = BLOCK_SIZE - pos
            # if the current file is full, close and open a new one.
            if remaining == 0:
                self._f.close()
                self._f = next(self._file_gen)
                pos, remaining = 0, BLOCK_SIZE
            self._f.write(b[:remaining])
            name = self._f.name if hasattr(self._f, 'name') else self._f._blob.name
            locs.append((name, pos))
            b = b[remaining:]
        return locs

    def close(self):
        self._f.close()

class MultiFileReader:
    """ Sequential binary reader of multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, bucket_name=None):
        self._base_dir = Path(base_dir)
        self._bucket = None if bucket_name is None else get_bucket(bucket_name)
        self._open_files = {}

    def read(self, locs, n_bytes):
        b = []
        for f_name, offset in locs:
            f_name = str(self._base_dir / f_name)
            if f_name not in self._open_files:
                self._open_files[f_name] = _open(f_name, 'rb', self._bucket)
            f = self._open_files[f_name]
            f.seek(offset)
            n_read = min(n_bytes, BLOCK_SIZE - offset)
            b.append(f.read(n_read))
            n_bytes -= n_read
        return b''.join(b)

    def close(self):
        for f in self._open_files.values():
            f.close()

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()
        return False

TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer


class InvertedIndex:
    def __init__(self, docs={}):
        """ Initializes the inverted index and add documents to it (if provided).
        Parameters:
        -----------
          docs: dict mapping doc_id to list of tokens
        """
        # stores document frequency per term
        self.df = Counter()
        # stores total frequency per term
        self.term_total = Counter()
        # stores posting list per term while building the index (internally),
        # otherwise too big to store in memory.
        self._posting_list = defaultdict(list)
        # mapping a term to posting file locations, which is a list of
        # (file_name, offset) pairs. Since posting lists are big we are going to
        # write them to disk and just save their location in this list. We are
        # using the MultiFileWriter helper class to write fixed-size files and store
        # for each term/posting list its list of locations. The offset represents
        # the number of bytes from the beginning of the file where the posting list
        # starts.
        self.posting_locs = defaultdict(list)
        self.n = len(docs)
        self.docs=list(docs.keys())
        self.nf={id:len(text) for id, text in docs.items()}

        for doc_id, tokens in docs.items():
            self.add_doc(doc_id, tokens)

    def calc_tf_idf(self, token, tf):
        df = self.df[token]
        n = self.n
        idf = math.log(n/df)
        return tf * idf

    def add_doc(self, doc_id, tokens):
        """ Adds a document to the index with a given `doc_id` and tokens. It counts
            the tf of tokens, then update the index (in memory, no storage
            side-effects).
        """
        w2cnt = Counter(tokens)
        self.term_total.update(w2cnt)
        for w, cnt in w2cnt.items():
            self.df[w] = self.df.get(w, 0) + 1
            tf_idf = self.calc_tf_idf(w, cnt)
            self._posting_list[w].append((doc_id, tf_idf))


    def write_index(self, base_dir, name, bucket_name=None):
        """ Write the in-memory index to disk. Results in the file:
            (1) `name`.pkl containing the global term stats (e.g. df).
        """
        #### GLOBAL DICTIONARIES ####
        self._write_globals(base_dir, name, bucket_name)

    def _write_globals(self, base_dir, name, bucket_name):
        path = str(Path(base_dir) / f'{name}.pkl')
        bucket = None if bucket_name is None else get_bucket(bucket_name)
        with _open(path, 'wb', bucket) as f:
            pickle.dump(self, f)

    def __getstate__(self):
        """ Modify how the object is pickled by removing the internal posting lists
            from the object's state dictionary.
        """
        state = self.__dict__.copy()
        del state['_posting_list']
        return state

    def posting_lists_iter(self, base_dir, bucket_name=None):
        """ A generator that reads one posting list from disk and yields
            a (word:str, [(doc_id:int, tf:int), ...]) tuple.
        """
        with closing(MultiFileReader(base_dir, bucket_name)) as reader:
            for w, locs in self.posting_locs.items():
                b = reader.read(locs, self.df[w] * TUPLE_SIZE)
                posting_list = []
                for i in range(self.df[w]):
                    doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
                    tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
                    posting_list.append((doc_id, tf))
                yield w, posting_list

    def read_a_posting_list(self, base_dir, w, bucket_name=None):
        posting_list = []
        if not w in self.posting_locs:
            return posting_list
        with closing(MultiFileReader(base_dir, bucket_name)) as reader:
            locs = self.posting_locs[w]
            b = reader.read(locs, self.df[w] * TUPLE_SIZE)
            for i in range(self.df[w]):
                doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
                tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
                posting_list.append((doc_id, tf))
        return posting_list

    @staticmethod
    def write_a_posting_list(b_w_pl, base_dir, bucket_name=None):
        posting_locs = defaultdict(list)
        bucket_id, list_w_pl = b_w_pl

        with closing(MultiFileWriter(base_dir, bucket_id, bucket_name)) as writer:
            for w, pl in list_w_pl:
                # convert to bytes
                b = b''.join([struct.pack('>If', doc_id, tf) for doc_id, tf in pl])

                # write to file(s)
                locs = writer.write(b)
                # save file locations to index
                posting_locs[w].extend(locs)
            path = str(Path(base_dir) / f'{bucket_id}_posting_locs.pickle')
            bucket = None if bucket_name is None else get_bucket(bucket_name)
            with _open(path, 'wb', bucket) as f:
                pickle.dump(posting_locs, f)
        return bucket_id


    @staticmethod
    def read_index(base_dir, name, bucket_name=None):
        path = str(Path(base_dir) / f'{name}.pkl')
        bucket = None if bucket_name is None else get_bucket(bucket_name)
        with _open(path, 'rb', bucket) as f:
            return pickle.load(f)

In [17]:
import struct

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){0,24}""", re.UNICODE)
def tokenize(text):
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
    return [token for token in tokens if token not in all_stopwords]

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    stemmer = PorterStemmer()
    for token in tokens:
        token=stemmer.stem(token)


    count = Counter([token for token in tokens if token not in all_stopwords])
    if len(count)==0:
        count = Counter([token for token in tokens])
    res = []
    for tup in count.items():
        res.append((stemmer.stem(tup[0]),(id,tup[1])))
    return res

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
      A sorted posting list.
    '''
    # YOUR CODE HERE
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
    return sorted_pl

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each element is a (token, df) pair.
    '''
    # YOUR CODE HERE
    return postings.map(lambda x: (x[0], len(x[1])))

from math import log

def calculate_tf_idf(postings, total_docs):
    ''' Calculates the TF-IDF for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, postings_list) pair, and
        postings_list is a list of (document_id, term_frequency) tuples for that token.
      total_docs: int
        The total number of documents in the corpus.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, [(document_id, tf_idf), ...]) pair.
    '''
    # Calculate IDF for each token
    # Note: log base is e by default, which is common in TF-IDF calculations
    def idf(df):
        return log(total_docs / df)
    
    # Calculate TF-IDF for each (document_id, term_frequency) tuple in the postings list
    def tf_idf(postings_list):
        df = len(postings_list)
        return [(doc_id, tf * idf(df)) for doc_id, tf in postings_list]
    
    return postings.map(lambda x: (x[0], tf_idf(x[1])))

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # YOUR CODE HERE
    buckets = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1]))).groupByKey().mapValues(list)
    res = buckets.map(lambda x: InvertedIndex().write_a_posting_list(x,"text/", bucket_name))
    return res

def partition_postings_and_write_title(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # YOUR CODE HERE
    buckets = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1]))).groupByKey().mapValues(list)
    res = buckets.map(lambda x: InvertedIndex().write_a_posting_list(x, "title/",bucket_name))
    return res

In [20]:
def group_by_b_id(items):
    grouped_dict = {}
    for b_id, item in items:
        if b_id not in grouped_dict:
            grouped_dict[b_id] = []
        grouped_dict[b_id].append(item)
    return [(b_id, grouped_dict[b_id]) for b_id in grouped_dict]




In [14]:
total_docs=doc_title_pairs.count()

In [55]:
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
title2df = calculate_tf_idf(postings_title, total_docs)
title2df_dict = title2df.collectAsMap()
_ = partition_postings_and_write_title(title2df).collect()

In [24]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [26]:
# Create inverted index instance
inverted_title = InvertedIndex(title2df_dict)
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_title.df = title2df_dict
# write the global stats out
inverted_title.write_index('.', 'index_title', bucket_name)
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

CommandException: No URLs matched: index_title.pkl


In [ ]:
for index, rdd in enumerate(text_splits):
    word_counts = rdd.flatMap(lambda x: word_count(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>70)
    # filtering postings and calculate df
    text2df = calculate_tf_idf(postings_filtered, total_docs)
    text2df_dict = text2df.collectAsMap()
    _ = partition_postings_and_write(text2df).collect()
    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='title'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
    # Create inverted index instance
    inverted_text = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted_text.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted_text.df = text2df_dict
    # write the global stats out
    inverted_text.write_index('.', f'index_text_part_{index}', bucket_name)
    # upload to gs
    index_src = f"index_text_part_{index}.pkl"
    index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst
    print(f'{index} is done')

CommandException: No URLs matched: index_text_part_0.pkl
0 is done


CommandException: No URLs matched: index_text_part_1.pkl
1 is done


CommandException: No URLs matched: index_text_part_2.pkl
2 is done


CommandException: No URLs matched: index_text_part_3.pkl
3 is done


CommandException: No URLs matched: index_text_part_4.pkl
4 is done


...

CommandException: No URLs matched: index_text_part_9.pkl
9 is done


CommandException: No URLs matched: index_text_part_10.pkl
10 is done


CommandException: No URLs matched: index_text_part_11.pkl
11 is done


CommandException: No URLs matched: index_text_part_12.pkl
12 is done


CommandException: No URLs matched: index_text_part_13.pkl
13 is done


In [45]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>70)
w2df_part = calculate_tf_idf(postings_filtered, total_docs)
_ = partition_postings_and_write(w2df_part)

In [47]:
partitioned_dct = w2df_part.mapPartitions(lambda iterator: dict(iterator)).collect()

Exception in thread "serve RDD 43" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [54]:
partitioned_dct

['cristian',
 'dörpfeld',
 'cookei',
 'insomuch',
 "cheese'",
 'lučko',
 'atraco',
 'ithamar',
 '1249',
 'vibia',
 'crosswhit',
 'jonk',
 'zwettl',
 'boudot',
 'marchon',
 '19541',
 'diver',
 'vo',
 'sirohi',
 'hoyland',
 'subaltern',
 'schine',
 'emigdio',
 "spear'",
 'polinski',
 'renaldi',
 'lajunen',
 'montyon',
 'insiem',
 'libertador',
 'sarva',
 'bonchurch',
 'paralyz',
 'kunsthistorisch',
 '9-e',
 'gap-fil',
 'wemp',
 'bouley',
 'waart',
 'miyaguchi',
 'schizoaffect',
 'matanović',
 'ōura',
 'frottag',
 'grandier',
 'remors',
 'fail-saf',
 'mcgee',
 'oolong',
 '1-meter',
 'stigliano',
 'beckley',
 'jombo',
 'kumit',
 'ន',
 'forestay',
 'shredder',
 'swelter',
 'pleasenc',
 'kuko',
 'oceanica',
 'gammaridea',
 '3954',
 'suppiluliuma',
 'd51',
 'willowi',
 "bolton'",
 'suntech',
 'tzedakah',
 'wherefor',
 "d'andr",
 'tether',
 'valedictorian',
 'frome',
 "gaozong'",
 'digital-to-analog',
 'kukë',
 'gilla',
 '40-day',
 'wringer',
 'public-interest',
 "o'mahony'",
 'supachai',
 'so

In [ ]:
for idx, dct in enumerate(partitioned_dct):
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f"text/part_{idx}"):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
    # Create inverted index instance
    inverted_text = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted_text.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted_text.df = dct
    # write the global stats out
    inverted_title.write_index('.', f'index_text_part_{idx}', bucket_name)
    # upload to gs
    index_src = f'index_text_part_{idx}.pkl'
    index_dst = f'gs://{bucket_name}/postings_gcp/text/{index_src}'
    !gsutil cp $index_src $index_dst

In [65]:
type(partitioned_dct[0])

str

In [ ]:
for idx, dct in enumerate(partitioned_dct):
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f"text/part_{idx}"):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
    # Create inverted index instance
    inverted_title = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted_title.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted_title.df = title2df_dict
    # write the global stats out
    inverted_title.write_index('.', f'index_text_part_{idx}', bucket_name)
    # upload to gs
    index_src = f'index_text_part_{idx}.pkl'
    index_dst = f'gs://{bucket_name}/postings_gcp/text/{index_src}'
    !gsutil cp $index_src $index_dst

In [49]:
import gc
def write_to_bucket(idx,rdd):
    dct = defaultdict(list)
    for key, value in rdd:
        dct[key].append(value)
    posting_locs_list = partition_postings_and_write(dct).collect()
    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
        for k, v in posting_loc.items():
            super_posting_locs[k].extend(v)
            
    # Create the storage client
    client = storage.Client()
    
    # Get the bucket
    bucket = client.get_bucket(bucket_name)
    
    # Define the destination blob
    blob_name = f"index_body_part{idx}.pkl"  # Adjust this as needed
    blob = bucket.blob(f"postings_gcp/{blob_name}")
    
    # Convert the dictionary to bytes using pickle
    data_bytes = pickle.dumps(super_posting_locs)
    
    # Upload the data to the blob
    blob.upload_from_string(data_bytes)
    
    del dct
    del super_posting_locs
    gc.collect()
    return None
w2df_part.ForEachPartition(write_to_bucket)

24/03/08 07:45:30 WARN YarnAllocator: Container from a bad node: container_1709823484704_0002_01_000009 on host: cluster-6670-w-1.c.gcp-413611.internal. Exit status: 143. Diagnostics: [2024-03-08 07:45:30.603]Container killed on request. Exit code is 143
[2024-03-08 07:45:30.605]Container exited with a non-zero exit code 143. 
[2024-03-08 07:45:30.605]Killed by external signal
.
24/03/08 07:45:30 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 6 for reason Container from a bad node: container_1709823484704_0002_01_000009 on host: cluster-6670-w-1.c.gcp-413611.internal. Exit status: 143. Diagnostics: [2024-03-08 07:45:30.603]Container killed on request. Exit code is 143
[2024-03-08 07:45:30.605]Container exited with a non-zero exit code 143. 
[2024-03-08 07:45:30.605]Killed by external signal
.
24/03/08 07:45:30 ERROR YarnScheduler: Lost executor 6 on cluster-6670-w-1.c.gcp-413611.internal: Container from a bad node: container_1709823484704_0002_01_

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 4 times, most recent failure: Lost task 0.3 in stage 14.0 (TID 517) (cluster-6670-w-1.c.gcp-413611.internal executor 12): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, it

In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>70)
w2df = calculate_tf_idf(postings_filtered, total_docs)

In [ ]:
w2df_dict = w2df.collectAsMap()
_ = partition_postings_and_write(postings_filtered).collect()

In [ ]:
# Create inverted index instance
inv_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inv_body.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inv_body.df = w2df_dict
# write the global stats out
inv_body.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
title_text = doc_title_pairs.map(lambda y: (y[1], tokenize(y[0]))).collect()

In [ ]:
body_text = doc_text_pairs.map(lambda y: (y[1], tokenize(y[0]))).collect()

In [ ]:
inv_title = InvertedIndex(dict(title_text))
inv_body = InvertedIndex(dict(body_text))

In [ ]:
def get_document_vectors(inverted_index, vocab):
    """
    This function retrieves document vectors from an inverted index with TF-IDF values.

    Args:
      inverted_index: A dictionary where keys are unique terms and values are postings lists.
          Each postings list is a list of tuples (document_id, tfidf_value).

    Returns:
      A dictionary where keys are document IDs and values are lists representing
          sparse document vectors (containing only non-zero TF-IDF values).
    """
    document_vectors = {}
    for term, postings_list in inverted_index._posting_list.items():
        for document_id, tfidf_value in postings_list:
          # Leverage document_id as the index for the sparse vector
            sparse_vec_indices = [vocab[term]]
            sparse_vec_values = [tfidf_value]
            # Update or initialize sparse vector for the document
            if document_id in document_vectors:
                  document_vectors[document_id].extend(zip(sparse_vec_indices, sparse_vec_values))
            else:
                  document_vectors[document_id] = list(zip(sparse_vec_indices, sparse_vec_values))
    # Convert the lists of indices and values to sparse vectors
    document_vectors = {doc_id: Vectors.sparse(len(vocab), [x[0] for x in lst], [x[1] for x in lst]) for doc_id, lst in document_vectors.items()}
    return document_vectors


In [ ]:
vocab_title = {term:x for x,term in enumerate(inv_title.term_total.keys())}
vocab_body = {term:x for x,term in enumerate(inv_body.term_total.keys())}

In [ ]:
doc_vecs_title = get_document_vectors(inv_title, vocab_title)
doc_vecs_body = get_document_vectors(inv_body, vocab_body)


In [ ]:
for doc in inv_title.docs:
    if doc not in doc_vecs_title.keys():
        doc_vecs_title[doc] = Vectors.sparse(len(vocab_title), [])
for doc in inv_body.docs:
    if doc not in doc_vecs_body.keys():
        doc_vecs_body[doc] =  Vectors.sparse(len(vocab_body), [])

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # YOUR CODE HERE
  # Convert to a set for faster lookup
  links = pages.flatMap(
        lambda page: [
            (page['id'], dest.id)  # Assuming dest_id is accessible as dest.id
            for dest in page['anchor_text']
        ]
    )

  # Remove duplicate links
  edges = links.distinct()

  # Extract vertices from the links
  # Vertices are unique ids present in either the source or destination of a link
  vertices = edges.flatMap(lambda link: [link[0], link[1]]).distinct().map(lambda id: (id,))
  return edges, vertices

In [ ]:
from graphframes import *

In [ ]:
edges, vertices = generate_graph(doc_anchor_pairs)
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('id').asc())

In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import MinMaxScaler

# Define a UDF to convert double to vector
to_vector_udf = udf(lambda d: Vectors.dense(d), VectorUDT())

# Apply the UDF to create a new column
pr = pr.withColumn("pagerank_vector", to_vector_udf(col("pagerank")))

# Now you can use the MinMaxScaler
scaler = MinMaxScaler(inputCol="pagerank_vector", outputCol="pagerank_normalized")

# Fit and transform the scaler
scaler_model = scaler.fit(pr)
pr_normalized = scaler_model.transform(pr).select("id", "pagerank_normalized")


In [ ]:
rdd = pr_normalized.rdd.map(lambda row: (row['id'], float(np.array(row['pagerank_normalized']))))

# Collect the RDD into a dictionary
pr_dict = rdd.collectAsMap()

In [ ]:
for doc in inv_title.docs:
    if doc not in pr_dict.keys():
        pr_dict[doc] = 0

In [ ]:
import numpy as np
def search(query):
    ''' Returns up to a 100 of your best search results for the query. This is
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each
        element is a tuple (wiki_id, title).
    '''
    # BEGIN SOLUTION
#     def load_item_from_pickle(filename):
#         with open(filename, 'rb') as file:
#           return pickle.load(file)
#     title_tf_idf=load_item_from_pickle('title_tf_idf.pkl')
#     text_tf_idf=load_item_from_pickle('text_tf_idf.pkl')
#     pagerank=load_item_from_pickle('pagerank.pkl')
#     allstopwords=load_item_from_pickle('allstopwords.pkl')
    def create_sparse_vector_from_counter(query_counter, vocab):

        """

        This function creates a sparse vector from a query represented as a counter.

        Args:
          query_counter: A Counter object representing the query, where keys are tokens and values are counts.
          vocab: A list of unique terms.

        Returns:
          A sparse vector representing the query, with non-zero counts at indices corresponding to tokens in the vocab.
        """
        # Initialize empty lists for indices and values
        sparse_vec_indices = []
        sparse_vec_values = []
        val_idx = {}
        # Iterate over tokens in the query counter
        for token, count in query_counter.items():
          # Get the index of the token in the vocab list
          if token in vocab:
            term_index = vocab[token]
            # Append the index and count to the sparse vector
#             sparse_vec_indices.append(term_index)
#             sparse_vec_values.append(count)
            val_idx[term_index]=count
        # Create a sparse vector from the indices and values
        val_idx = dict(sorted(val_idx.items(), key=lambda x: x[0]))
        sparse_vector = Vectors.sparse(len(vocab), list(val_idx.keys()), list(val_idx.values()))

        return sparse_vector


    np.seterr(divide='ignore', invalid='ignore')
    stemmer=PorterStemmer()
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    tokens = [stemmer.stem(token.group()) if len(token.group()) > 1 else token.group() for token in RE_WORD.finditer(query.lower())]
    count = Counter([token for token in tokens])
    q_vec_title = create_sparse_vector_from_counter(count,vocab_title)
    q_vec_body = create_sparse_vector_from_counter(count,vocab_body)
    def cosine_similarity(vec, column_vector):
        """
        Calculate the cosine similarity between two vectors.
        """
        dot_product = vec.dot(column_vector)
        norm_vec = vec.norm(2)
        norm_column_vector = column_vector.norm(2)
        if norm_vec == 0 or norm_column_vector == 0:
            similarity = 0
        else:
            similarity = dot_product / (norm_vec * norm_column_vector)
        return similarity
    res = []
    for title, body in zip(doc_vecs_title.items(),doc_vecs_body.items()):
        id = title[0]
        vec_title = title[1]
        vec_body = body[1]
        sim_title = cosine_similarity(q_vec_title,vec_title)
        sim_body = cosine_similarity(q_vec_body,vec_body)
        pr = pr_dict[id]
        res.append((id, sim_title, sim_body, pr))
    res = [x if not (np.isnan(x[1]) or np.isnan(x[2])) else (x[0],0.0,0.0, x[3]) for x in res]
    return [x[0] for x in sorted(res, key=lambda x: 0.5*x[1]+0.3*x[2]+0.2*x[3], reverse=True)[:100]]

In [ ]:
search("fast car")